In [ ]:
import yfinance as yf
from statsmodels.tsa.stattools import adfuller
import pandas as pd

# Define the list of forex pairs to test
# The '=X' suffix is necessary for forex data on Yahoo Finance
forex_pairs = ['AUDUSD=X', 'EURGBP=X', 'NZDUSD=X', 'USDCHF=X',
               'AUDJPY=X', 'CADJPY=X', 'EURUSD=X', 'GBPUSD=X',
               'NZDJPY=X', 'USDCAD=X', 'CHFJPY=X', 'EURJPY=X',
               'EURNZD=X', 'GBPJPY=X', 'USDJPY=X', 'GC=F']

# Define the time period for the data
# Using a recent 2-year period is a good starting point for mean reversion tests
start_date = '2015-01-01'
end_date = '2025-01-01'

# List to store the results for each pair
results = []

print(f"--- Running ADF Test on {len(forex_pairs)} Pairs ---")
print(f"Data Period: {start_date} to {end_date}\n")

for pair in forex_pairs:
    try:
        data = yf.download(pair, start=start_date, end=end_date)
        if data.empty:
            print(f"Warning: No data found for {pair}. Skipping...")
            continue

        # Use the 'Close' price and remove any missing values
        price_series = data['Close'].dropna()

        # Run the ADF test
        adf_result = adfuller(price_series)

        # Store the results in a dictionary
        results.append({
            'Pair': pair,
            'ADF Statistic': adf_result[0],
            'p-value': adf_result[1]
        })

    except Exception as e:
        print(f"An error occurred for {pair}: {e}. Skipping...")
        continue

# Sort the results from most to least mean-reverting
# The ADF Statistic is negative, so we sort in ascending order (most negative first)
results.sort(key=lambda x: x['ADF Statistic'])

print("--- Ranked Results (Most Mean-Reverting to Least) ---")
print("{:<12} {:<15} {:<10}".format('Pair', 'ADF Statistic', 'p-value'))
print("-" * 37)

for r in results:
    print("{:<12} {:<15.4f} {:<10.4f}".format(r['Pair'], r['ADF Statistic'], r['p-value']))